In [1]:
import pandas as pd
import glob, os
import numpy as np
import matplotlib.pyplot as plt

## Source
- [Houston Police Department Crime Statistics](http://www.houstontx.gov/police/cs/crime-stats-archives.htm)
	- years: 2008 - 2017
	- format: Access or Excel

In [2]:
ls crime_data/2009

aug09.mdb*  dec09.xls*  jul09.xls*  nov09.mdb*  oct09.xls*
aug09.xls*  jan09.mdb*  jun09.mdb*  nov09.xls*  sep09.mdb*
dec09.mdb*  jul09.mdb*  jun09.xls*  oct09.mdb*  sep09.xls*


## combine all files into one dataframe

In [ ]:
path = 'crime_data/2009'
all_files = glob.glob(os.path.join(path, "*.xls")) 

df_from_each_file = (pd.read_excel(f) for f in all_files)
df   = pd.concat(df_from_each_file, ignore_index=True)

In [ ]:
df.info()

In [ ]:
df.head(5)

## Lets create a copy

In [ ]:
df1 = df.copy()

In [ ]:
df1.columns

In [ ]:
df1.head(5)

In [ ]:
df1['Unnamed: 1'].value_counts(dropna=False)

In [ ]:
df1['Field11'].value_counts(dropna=False)

## Rename columns

In [ ]:
df1.rename(columns={
    '# Of Offenses': '# offenses',
    'Block Range': 'BlockRange',
    'Street Name': 'StreetName',}, inplace=True)

In [ ]:
df1.info()

In [ ]:
df1.head(5)

## create a subdataframe with the columns that we want

In [ ]:
df10 = df1[['Date','Beat','BlockRange','StreetName','Offense Type','Premise','# offenses','Hour']]

In [ ]:
df10.info()

In [ ]:
df10.tail()

## Now we can inspect df

In [ ]:
df10.Date.unique() ## timestamp

In [ ]:
df10.Beat.unique()  ##

In [ ]:
df10.Beat.value_counts(dropna=False)  # UNK : 229

In [ ]:
df10.BlockRange.unique()  #

In [ ]:
df10.BlockRange.value_counts(dropna=False)  #

In [ ]:
df10.StreetName.unique()  #

In [ ]:
df10.StreetName.value_counts(dropna=False)  # 

In [ ]:
df10['Offense Type'].unique() 

In [ ]:
df10['Offense Type'].value_counts(dropna=False) # good, that its clean not that there were 73591 thefts!! 

In [ ]:
df10.Premise.unique()  # lots of weird values

In [ ]:
df10.Premise.value_counts(dropna=False)  # srip extra spaces

In [ ]:
df10['# offenses'].unique()

In [ ]:
df10['# offenses'].value_counts(dropna=False)  #

In [ ]:
df10.Hour.unique()

In [ ]:
df10.Hour.value_counts(dropna=False)

# Cleanup
## Premise Column

- strip empty spaces :not needed

In [ ]:
len(df10.Premise.unique())

In [ ]:
df10['Premise'] = df10['Premise'].str.strip()

In [ ]:
len(df10.Premise.unique())

In [ ]:
df10.Premise.value_counts(dropna=False)

# Cleanup
## Offense Type Column

preatty clean!


In [ ]:
df10['Offense Type'].value_counts(dropna=False)

In [ ]:
df10['Offense Type'].unique()

In [ ]:
len(df10['Offense Type'].unique())

In [ ]:
df10['Offense Type'] = df10['Offense Type'].str.strip()

In [ ]:
df10['Offense Type'].value_counts(dropna=False)

In [ ]:
len(df10['Offense Type'].unique())

# Cleanup
## StreetName Column

preatty clean

In [ ]:
df10.StreetName.value_counts(dropna=False).head()

In [ ]:
len(df10.StreetName.unique())

In [ ]:
df10['StreetName'] = df10['StreetName'].str.strip()

In [ ]:
len(df10.StreetName.unique())

In [ ]:
df10[df10.StreetName.isnull()].head()

In [ ]:
df10.StreetName.value_counts(dropna=False).head()

# Cleanup
## BlockRange Column

- create mask to find 'UNK' values
- match with similar beat value (Needs to be done)

In [ ]:
df10.BlockRange.value_counts(dropna=False).head()  # find UNK

In [ ]:
unk = df10.BlockRange == 'UNK'  # boolean mask

In [ ]:
df10[unk]  # 0 rows

In [ ]:
df10[df10.BlockRange.isnull()].head()

# Cleanup
## Beat Column

preatty clean

In [ ]:
df10.Beat.unique()

In [ ]:
len(df10.Beat.unique())

In [ ]:
df10['Beat'] = df10['Beat'].str.strip()

In [ ]:
len(df10.Beat.unique())

In [ ]:
df10[df10.Beat.isnull()].head()

In [ ]:
df10.Beat.value_counts(dropna=False).head()

In [ ]:
df10.info()

## Cleanup

### Date column
- convert to datetime
- index date colimn
- sort index

In [ ]:
df10.head(5)

In [ ]:
df10['Date'] = pd.to_datetime(df10['Date'])

df10 = df10.set_index('Date').sort_index(ascending=True)

In [ ]:
df10.head(5)

In [ ]:
df10.info()

## Odd dates
`DatetimeIndex: 136263 entries, 1959-06-06 to NaT
`
- some values are not from this year, lets look

In [ ]:
df2010 = df10.loc['2010-01-01':'2010-12-31']  # rows with date from 01,01,10 - 12,31,10

In [ ]:
df2010_wrong_date = df10[:"2009"]  # rows with year 0  upto 2010
df2010_wrong_date.info()

In [ ]:
df2010.info()

# NAN values

- Beat: 0
- BlockRange: 0
- StreetName: 0
- Offense Type: 0
- Premise: 0
- Hour: 0

In [ ]:
beat_nan = df2010.Beat.isnull()
block_nan = df2010.BlockRange.isnull()
str_nan = df2010.StreetName.isnull()
off_nan = df2010['Offense Type'].isnull()
premise_nan = df2010.Premise.isnull()  #
hour_nan = df2010.Hour.isnull()  #

In [ ]:
df2010[beat_nan]

In [ ]:
df2010[block_nan]

In [ ]:
df2010[str_nan]

In [ ]:
df2010[off_nan]

In [ ]:
df2010[hour_nan]

## Save clean data to  to csv

In [ ]:
df2010.to_csv('clean_data/crime_data/crime10_clean.csv')

In [ ]:
ls clean_data/crime_data/

## DROP nan
drop nan values of StreetName

In [ ]:
df2017.head()

In [ ]:
df2017 = df2017.dropna(subset=['StreetName'])  # drop nan values from StreetName, 8 rows

In [ ]:
df2017.info()

In [ ]:
df2017['Premise'] = df2017['Premise'].fillna('unk')

In [ ]:
df2017.info()

In [ ]:
df2017['Offense Type'].value_counts(dropna=False)

## Fillna
will use this since Dont know what to do... :/

In [ ]:
df2017['Offense Type'].fillna(method='ffill', inplace=True)

In [ ]:
df2017.info()

## Save clean data to  to csv

In [ ]:
df2017.to_csv('clean_data/crime_data/crime17_clean.csv')

In [ ]:
ls clean_data/crime_data/